# 06 — Basic Neural Network (Multi-class Synthetic)
**Goal:** Build and train a simple MLP for 3-class classification on synthetic blobs; visualize decision boundary.


In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
torch.manual_seed(0)

In [ ]:
# Data (3 classes)
X, y = make_blobs(n_samples=1500, centers=3, n_features=2, cluster_std=1.5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test_np = y_test  # numpy copy
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# MLP
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2,32), nn.ReLU(),
            nn.Linear(32,32), nn.ReLU(),
            nn.Linear(32,3)
        )
    def forward(self,x): return self.net(x)

model = MLP()
opt = optim.Adam(model.parameters(), lr=0.01)
crit = nn.CrossEntropyLoss()

losses=[]
for ep in range(200):
    opt.zero_grad()
    out = model(X_train)
    loss = crit(out, y_train)
    loss.backward()
    opt.step()
    losses.append(loss.item())

with torch.no_grad():
    preds = model(X_test).argmax(1).numpy()
acc = accuracy_score(y_test_np, preds)
print(f'Test Accuracy: {acc:.3f}')

In [ ]:
# Plot loss
plt.plot(losses); plt.title('Training Loss'); plt.xlabel('epoch'); plt.ylabel('CE Loss'); plt.show()

In [ ]:
# Decision boundary visualization
xx, yy = np.meshgrid(np.linspace(X[:,0].min()-1, X[:,0].max()+1, 200),
                     np.linspace(X[:,1].min()-1, X[:,1].max()+1, 200))
grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
with torch.no_grad():
    Z = model(grid).argmax(1).numpy().reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, s=10, cmap='viridis')
plt.title('Decision Regions (Train)'); plt.show()

## 🧩 Student Practice 1 — Add Dropout
- Insert `nn.Dropout(0.3)` between layers; measure accuracy change.


In [ ]:
# TODO: Modify the MLP to include Dropout and retrain


## ⚙️ Student Practice 2 — Activation Experiment
- Replace ReLU with Tanh; discuss convergence/accuracy differences.

In [ ]:
# TODO: Change activations to nn.Tanh() and compare
